# Catch me if you can ("Alice") competition
https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/overview


In [1]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [2]:
#getting the data
df_train = pd.read_csv('data/train_sessions.csv')
df_test = pd.read_csv('data/test_sessions.csv')

Exploring the data

In [3]:
df_train.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253561 entries, 0 to 253560
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   session_id  253561 non-null  int64  
 1   site1       253561 non-null  int64  
 2   time1       253561 non-null  object 
 3   site2       250098 non-null  float64
 4   time2       250098 non-null  object 
 5   site3       246919 non-null  float64
 6   time3       246919 non-null  object 
 7   site4       244321 non-null  float64
 8   time4       244321 non-null  object 
 9   site5       241829 non-null  float64
 10  time5       241829 non-null  object 
 11  site6       239495 non-null  float64
 12  time6       239495 non-null  object 
 13  site7       237297 non-null  float64
 14  time7       237297 non-null  object 
 15  site8       235224 non-null  float64
 16  time8       235224 non-null  object 
 17  site9       233084 non-null  float64
 18  time9       233084 non-null  object 
 19  si

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82797 entries, 0 to 82796
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   session_id  82797 non-null  int64  
 1   site1       82797 non-null  int64  
 2   time1       82797 non-null  object 
 3   site2       81308 non-null  float64
 4   time2       81308 non-null  object 
 5   site3       80075 non-null  float64
 6   time3       80075 non-null  object 
 7   site4       79182 non-null  float64
 8   time4       79182 non-null  object 
 9   site5       78341 non-null  float64
 10  time5       78341 non-null  object 
 11  site6       77566 non-null  float64
 12  time6       77566 non-null  object 
 13  site7       76840 non-null  float64
 14  time7       76840 non-null  object 
 15  site8       76151 non-null  float64
 16  time8       76151 non-null  object 
 17  site9       75484 non-null  float64
 18  time9       75484 non-null  object 
 19  site10      74806 non-nul

We do have quite a lot of null values plus the date time features dont have the correct type

In [6]:
def change_dtype(df):
    new_df = pd.DataFrame()
    for i in range(len(df.columns)):
        if 'time' in df.columns[i]:
            df[df.columns[i]] = pd.to_datetime(df[df.columns[i]])
            new_df[f'{df.columns[i]}_year'] = df[df.columns[i]].dt.year
            new_df[f'{df.columns[i]}_month'] = df[df.columns[i]].dt.month
            new_df[f'{df.columns[i]}_day'] = df[df.columns[i]].dt.day
            new_df[f'{df.columns[i]}_hour'] = df[df.columns[i]].dt.hour
            new_df[f'{df.columns[i]}_minute'] = df[df.columns[i]].dt.minute

        else:
            new_df[df.columns[i]] = df[df.columns[i]]
    
    return new_df

df_train_rf = change_dtype(df_train)
df_test_rf = change_dtype(df_test)
df_train.dtypes

session_id             int64
site1                  int64
time1         datetime64[ns]
site2                float64
time2         datetime64[ns]
site3                float64
time3         datetime64[ns]
site4                float64
time4         datetime64[ns]
site5                float64
time5         datetime64[ns]
site6                float64
time6         datetime64[ns]
site7                float64
time7         datetime64[ns]
site8                float64
time8         datetime64[ns]
site9                float64
time9         datetime64[ns]
site10               float64
time10        datetime64[ns]
target                 int64
dtype: object

In [7]:
df_train_rf.columns

Index(['session_id', 'site1', 'time1_year', 'time1_month', 'time1_day',
       'time1_hour', 'time1_minute', 'site2', 'time2_year', 'time2_month',
       'time2_day', 'time2_hour', 'time2_minute', 'site3', 'time3_year',
       'time3_month', 'time3_day', 'time3_hour', 'time3_minute', 'site4',
       'time4_year', 'time4_month', 'time4_day', 'time4_hour', 'time4_minute',
       'site5', 'time5_year', 'time5_month', 'time5_day', 'time5_hour',
       'time5_minute', 'site6', 'time6_year', 'time6_month', 'time6_day',
       'time6_hour', 'time6_minute', 'site7', 'time7_year', 'time7_month',
       'time7_day', 'time7_hour', 'time7_minute', 'site8', 'time8_year',
       'time8_month', 'time8_day', 'time8_hour', 'time8_minute', 'site9',
       'time9_year', 'time9_month', 'time9_day', 'time9_hour', 'time9_minute',
       'site10', 'time10_year', 'time10_month', 'time10_day', 'time10_hour',
       'time10_minute', 'target'],
      dtype='object')

In [8]:
df_test_rf.columns

Index(['session_id', 'site1', 'time1_year', 'time1_month', 'time1_day',
       'time1_hour', 'time1_minute', 'site2', 'time2_year', 'time2_month',
       'time2_day', 'time2_hour', 'time2_minute', 'site3', 'time3_year',
       'time3_month', 'time3_day', 'time3_hour', 'time3_minute', 'site4',
       'time4_year', 'time4_month', 'time4_day', 'time4_hour', 'time4_minute',
       'site5', 'time5_year', 'time5_month', 'time5_day', 'time5_hour',
       'time5_minute', 'site6', 'time6_year', 'time6_month', 'time6_day',
       'time6_hour', 'time6_minute', 'site7', 'time7_year', 'time7_month',
       'time7_day', 'time7_hour', 'time7_minute', 'site8', 'time8_year',
       'time8_month', 'time8_day', 'time8_hour', 'time8_minute', 'site9',
       'time9_year', 'time9_month', 'time9_day', 'time9_hour', 'time9_minute',
       'site10', 'time10_year', 'time10_month', 'time10_day', 'time10_hour',
       'time10_minute'],
      dtype='object')

In [9]:
set(df_train_rf.columns) - set(df_test_rf.columns)

{'target'}

In [10]:
target_visual = df_train['target'].value_counts()
target_visual

target
0    251264
1      2297
Name: count, dtype: int64

Only a very small number of data is alice

In [11]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf

RandomForestClassifier(random_state=42)

In [12]:
parameters_rf = {'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5,10,15]}

In [13]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [15]:
grid_rf = GridSearchCV(rf, parameters_rf, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_rf.fit(df_train_rf.drop(['target'], axis=1), df_train_rf['target'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15], 'max_features': [1, 2, 4],
                         'min_samples_leaf': [3, 5, 7, 9]},
             scoring='roc_auc', verbose=1)

In [16]:
grid_rf.best_params_, grid_rf.best_score_

({'max_depth': 15, 'max_features': 4, 'min_samples_leaf': 3},
 np.float64(0.9950619470416033))

In [18]:
best_rf = grid_rf.best_estimator_

y_pred = best_rf.predict(df_test_rf)
y_pred_proba = best_rf.predict_proba(df_test_rf)  
